In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np

import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [219]:
complete_df = get_data(url)
complete_df = preprocess(df)

In [242]:
complete_df

,Comunidad Autónoma,Fecha,Casos,Hospitalizados,UCI,Fallecidos
0,Andalucía,2020-02-20,0,0,0,0
1,Aragón,2020-02-20,0,0,0,0
2,Principado de Asturias,2020-02-20,0,0,0,0
3,Islas Baleares,2020-02-20,1,0,0,0
4,Canarias,2020-02-20,1,0,0,0
...,...,...,...,...,...,...
698,Melilla,2020-03-27,45,22,2,1
699,Murcia,2020-03-27,802,231,58,17
700,Comunidad Foral de Navarra,2020-03-27,1829,827,84,70
701,País Vasco,2020-03-27,5136,2612,176,221


In [269]:
daily_cases = complete_df.groupby('Fecha').sum()

daily_cases = daily_cases.reindex(index=daily_cases.index[::-1])

table = hv.Table(daily_cases, kdims='Fecha', vdims=list(daily_cases.columns)[0:])

In [270]:
render(table, 'table')

current day 

In [240]:
date = datetime.today()

In [220]:
values = df[df['Fecha'] == f'{date.year}-{date.month}-{date.day-1}']

In [221]:
values['Casos'].sum()

72248

In [208]:
date = datetime.today()
values = df[df['Fecha'] == f'{date.year}-{date.month}-{date.day}'].values

if values.size > 0:
    total = 

test

In [302]:
url = 'https://covid19.isciii.es/resources/serie_historica_acumulados.csv'
width = 900

def get_data(csv_url):
    try:
        df = pd.read_csv(csv_url, engine='python', skipfooter=1)
        df.to_csv('COVID19_Spain.csv', index=False)
    except URLError:
        df = pd.read_csv('COVID19_Spain.csv')
    return df


def preprocess(df):
    df.rename(columns={'CCAA Codigo ISO': 'Comunidad Autónoma'}, inplace=True)
    df.columns = map(str.rstrip, df.columns)
    
    # Avoiding user's input mistakes
    df.fillna(value=0, inplace=True)
    # Removing characters from columns
    for col in df.columns[-4:]:
        df[col] = df[col].astype(str)
        df[col] = df.loc[:, col].str.replace(r'\.0', '')
        df[col] = df.loc[:, col].str.replace(r'\D', '')
        df[col] = df.loc[:, col].astype(int)
    
    df.Fecha = pd.to_datetime(df.Fecha, format='%d/%m/%Y')
#     df.Fecha = df.loc[:, 'Fecha'].dt.strftime('%d-%m')

    # Autonomous communities names
    tag_list = ['MD', 'CT', 'PV', 'CL', 'CM', 'AN', 'VC', 'GA', 'NC', 'AR', 'RI', 'EX', 'AS', 'CN', 'CB', 'IB', 'MC',
                'ME', 'CE']
    name_list = ['Madrid', 'Cataluña', 'País Vasco', 'Castilla y León', 'Castilla-La Mancha', 'Andalucía',
                 'Comunidad Valenciana', 'Galicia', 'Comunidad Foral de Navarra', 'Aragón', 'La Rioja', 'Extremadura',
                 'Principado de Asturias', 'Canarias', 'Cantabria', 'Islas Baleares', 'Murcia', 'Melilla', 'Ceuta']
    df.replace(tag_list, name_list, inplace=True)

    return df

def render(figure, name):
    renderer = hv.renderer('bokeh')
    renderer.save(figure, f'{name}.html')


def get_acc_table(complete_df):
    daily_cases = complete_df.groupby('Fecha').sum()
    daily_cases = daily_cases.reindex(index=daily_cases.index[::-1])
    table = hv.Table(daily_cases, kdims='Fecha', vdims=list(daily_cases.columns)[0:])
    table.opts(width=width)
    return table
    
    
def get_ccaa(complete_df):
    # preprocessing
    df = complete_df[['Comunidad Autónoma', 'Fecha', 'Casos']]

    df = df.sort_values(by='Casos', ascending=False)
    df.reset_index(drop=True, inplace=True)

    # plot
    # for better date representation
    df.Fecha = df.loc[:, 'Fecha'].dt.strftime('%d-%m')

    color_cycle = hv.Palette('Inferno')
    opts_dict = dict(color=color_cycle, width=width, height=450, xrotation=90, ylabel='Casos', xlabel='Fecha', invert_xaxis=True, shared_axes=False,
                  tools=['hover'], stacked=True, show_legend=False, colorbar=True, line_color='white', show_grid=True, title='Escala lineal')
    
    bars = hv.Bars(df, kdims=['Fecha', 'Comunidad Autónoma'], vdims='Casos')
    bars.opts(**opts_dict)
    
    log_bars = hv.Bars(df, kdims=['Fecha', 'Comunidad Autónoma'], vdims='Casos')
    
    opts_dict['logy'] = True
    opts_dict['title'] = 'Escala logarítmica'
    log_bars.opts(**opts_dict)
    
    layout = (bars * log_bars)
    layout.opts(tabs=True, title='Casos por comunidad')
    
    return layout


def get_ccaa_heatmap(complete_df):
    df = complete_df[['Comunidad Autónoma', 'Fecha', 'Casos']]
    df = df.sort_values(by=['Comunidad Autónoma', 'Fecha'], ascending=[False, True])
    df.Fecha = df.loc[:, 'Fecha'].dt.strftime('%d-%m')

    heatmap = hv.HeatMap(df, kdims=['Fecha', 'Comunidad Autónoma'], vdims='Casos')
    heatmap.opts(width=width, height=500, tools=['hover'], logz=True, title='Casos acumulados',
                 toolbar='above', xaxis=None, xlabel='', ylabel='',
                 colorbar=True, clim=(1, np.nan))
    return heatmap


def get_split(complete_df):
    # preprocessing
    df = complete_df.copy()
    df['Leves'] = abs(df['Casos'] - (df['Hospitalizados'] + df['UCI']))
    df = df[['Comunidad Autónoma', 'Fecha', 'Leves', 'Hospitalizados', 'UCI', 'Fallecidos']]
    
    df = pd.melt(df, id_vars=df.columns[:2], var_name='Estado', value_name='Número')

    # plot
    key_dimensions = ['Fecha', 'Estado']
    value_dimensions = ['Número']
    
    # for better date representation
    df.Fecha = df.loc[:, 'Fecha'].dt.strftime('%d-%m')
    
    macro = hv.Table(df, key_dimensions, value_dimensions)
    fig = macro.to.bars(key_dimensions, value_dimensions, 'Comunidad Autónoma')

    color_cycle = hv.Cycle(['#FEE5AD', '#F7A541', '#F45D4C', '#2E2633'])
    fig.opts(
        opts.Bars(color=color_cycle, show_legend=True, stacked=True, aspect=16 / 9,
                  ylabel='', xlabel='Fecha', responsive=True, tools=['hover'], title='Desglose por estado',
                  legend_position='top_left', xrotation=90, active_tools=['pan', 'wheel_zoom']),
    )
    return fig


def get_daily_increment(complete_df):
    df = complete_df.copy()
    
    # preprocessing
    # New columns
    for column in df.columns[-4:]:
        df[f'Incremento porcentual en {column}'] = ''
        df[f'Incremento diario en {column}'] = ''

    key_values = ['Casos', 'Hospitalizados', 'UCI', 'Fallecidos']
    for ccaa in df['Comunidad Autónoma'].unique():
        for value in key_values:
            before = df.loc[df['Comunidad Autónoma'] == ccaa, :][value][1:]
            after = df.loc[df['Comunidad Autónoma'] == ccaa, :][value][:-1]
            difference = before.values - after.values
            difference = np.append(difference, 0)
            
            # daily increment
            df.loc[df['Comunidad Autónoma'] == ccaa, f'Incremento diario en {value}'] = difference
            # daily percentual increment, you have to replace the zeros for ones in order to divide properly
            divisor = df.loc[df['Comunidad Autónoma'] == ccaa, :][value][:].values
            divisor = np.where(divisor == 0, 1, divisor)
            
            increment = difference / divisor
            df.loc[df['Comunidad Autónoma'] == ccaa, f'Incremento porcentual en {value}'] = increment
    
    df = pd.concat([df[['Comunidad Autónoma', 'Fecha']], df.filter(regex='Incremento')], axis=1)
    df = pd.melt(df, id_vars=df.columns[:2], var_name='Estado', value_name='Incremento')
    
    # plot
    key_dimensions = ['Fecha', 'Estado']
    value_dimensions = ['Incremento']
    
    # for better date representation
    df.Fecha = df.loc[:, 'Fecha'].dt.strftime('%d-%m')
    
    # barplot
#     macro = hv.Table(df, key_dimensions, value_dimensions)
#     fig = macro.to.bars(key_dimensions, value_dimensions, ['Comunidad Autónoma', 'Estado'])
    
#     fig.opts(
#         opts.Bars(show_legend=True, stacked=True, aspect=16 / 9,
#                   ylabel='', xlabel='Fecha', responsive=True, tools=['hover'], title='Incremento',
#                   legend_position='top_left', xrotation=90, active_tools=['pan', 'wheel_zoom']),
#     )
    
    # heatmap
    
    
    return fig
    

def get_dashboard(csv_url):
    # obtaining data
    df = get_data(csv_url)
    
    # preprocessing
    df = preprocess(df)
    
    # all cases
    heatmap = get_ccaa_heatmap(df)
    fig_global = get_ccaa(df)
    table = get_acc_table(df)
    
    layout = (heatmap + fig_global * table).cols(1)
    render(layout, 'dashboard')


def generate_html(csv_url):
    get_dashboard(csv_url)

In [303]:
generate_html(url)

In [296]:
hv.help(hv.Overlay)
hv.Overlay

Overlay

Online example: http://holoviews.org/reference/containers/bokeh/Overlay.html

-------------
Style Options
-------------

	background_fill_alpha, background_fill_color, border_alpha, border_color, border_hover_alpha, border_hover_color, border_hover_line_alpha, border_hover_line_color, border_line_alpha, border_line_cap, border_line_color, border_line_dash, border_line_join, border_line_width, border_muted_alpha, border_muted_color, border_muted_line_alpha, border_muted_line_color, border_nonselection_alpha, border_nonselection_color, border_nonselection_line_alpha, border_nonselection_line_color, border_selection_alpha, border_selection_color, border_selection_line_alpha, border_selection_line_color, click_policy, glyph_height, glyph_width, label_height, label_standoff, label_width, legend_padding, legend_spacing, text_align, text_alpha, text_baseline, text_color, text_font, text_font_size, text_font_style

(Consult bokeh's documentation for more information.)

------------
Pl

Composing

In [168]:
from bs4 import BeautifulSoup

In [179]:
soup_heatmap = BeautifulSoup(open('heatmap.html'))

In [180]:
soup_bars = BeautifulSoup(open('bars.html'))

In [184]:
for element in soup_bars.body:
    soup_heatmap.body.append(element)

1
1


In [182]:
with open('dashboard.html', 'w') as file:
    file.write(str(soup_heatmap))